In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(pd.options.display.max_rows)
pd.set_option("display.max_columns", None)  # print all columns of a dataframe; we can replace "None" by any number if we want to print a certain number of columns
pd.set_option("max_colwidth", None)   # Set the Column Width.
pd.set_option("display.max_rows", 100) 

2.0.3
60


In [ ]:
#################  Size of the Original Sales files  ################# 
# Total: 830400 rows × 569 columns
# Germany: 330955 rows × 569 columns
# United kingdom: 260410 rows × 569 columns
# France: 239035 rows × 569 columns

# Combine datasets in 3 countries

In [ ]:
### Load single sales data ###
# Mac
df_uk_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_uk.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_fr_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_fr.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_ge_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_ge.csv', encoding='unicode_escape',  engine='python')

# rename the columns 'Country/Territory-Name' or 'ï»¿c' to 'country'
df_uk_sales.rename(columns={'Country/Territory-Name':'country'}, inplace=True)
df_fr_sales.rename(columns={'Country/Territory-Name':'country'}, inplace=True)
df_ge_sales.rename(columns={'ï»¿c':'country'}, inplace=True)

# concatenate the three dataframes horizontally
df = pd.concat([df_uk_sales, df_fr_sales, df_ge_sales],axis=0, ignore_index=True)

# Delete rows with zero sales in year 2015-2023

In [ ]:
# drop rows with no sales in years 2015 to 2023
df = df[(df.loc[:,'2015/01':'2023/12']!=0).any(axis=1)]

# drop columns '1981/01' to '2014/12' and '2024/01' to '2024/06':
df = df.drop(df.loc[:, '1981/01':'2014/12'].columns, axis = 1)
df = df.drop(df.loc[:, '2024/01':'2024/06'].columns, axis = 1)

# Aggregate monthly sales to annual sales

In [ ]:
# aggregate from monthly sales to annuals sales
for year in range(2015, 2024):
    month_columns = [f"{year}/{month:02d}" for month in range(1, 13)]  # this create a list of column names for the 12 months of the year, useful skill!
    df[str(year)] = df[month_columns].sum(axis=1, skipna=True)         # Sum the monthly values for each year, handling missing columns
df.drop(columns=df.filter(like='/'), inplace=True)                     # Drop the original monthly columns        # columns=df_s.filter(like='/') pick the columns that contain '/', useful skill!

In [ ]:
### Save to a csv file
df.to_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_all.csv', index=False) 

# Aggregate duplicates

In [ ]:
# Mac
df = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_all.csv', low_memory=False)
# Windows
#df = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/data/processed data/sales/2015_2023_sales_all.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')

In [ ]:
### Show the number of sales in each year before aggregate duplicates
#df['2019'].sum()   # 8,126,958
#df['2020'].sum()   # 6,195,737
#df['2021'].sum()   # 5,924,511
#df['2022'].sum()   # 5,793,956
#df['2023'].sum()   # 6,519,213

# show the number of sales in 2023 for each country
#df[df['country']== 'Germany']['2023'].sum()   # 2,844,609
#df[df['country']== 'United Kingdom']['2023'].sum()   # 1,903,054
#df[df['country']== 'France']['2023'].sum()   # 1,771,550

1771550

In [ ]:
# check the column types of df
df.dtypes

# remove the empty space at the beginning and the end of the string
df['Price. (local)'] = df['Price. (local)'].str.strip() 
df['Price. (EUR)'] = df['Price. (EUR)'].str.strip()

# for column 'Price. (local)', 'Price. (EUR), replace 'Unspecified' with NaN
df['Price. (local)'] = df['Price. (local)'].replace('Unspecified', np.nan)
df['Price. (EUR)'] = df['Price. (EUR)'].replace('Unspecified', np.nan)

# change the column 'Price. (local)' to float
df['Price. (local)'] = df['Price. (local)'].astype(float)
df['Price. (EUR)'] = df['Price. (EUR)'].astype(float)

In [ ]:
### check for potential duplicates

# create the list of column for grouping
df_select = df.drop(df.loc[:, 'Price. (local)' : '2023'].columns, axis = 1)  
df['duplicate'] = df_select.groupby(df_select.columns.tolist(), dropna=False).transform('size')     # option: grouped by selected columns: list(df_select.columns[1:])
df_sort = df.sort_values(by = ['duplicate'] + df_select.columns.tolist(), ascending = [False] + [True]*(len(df_select.columns)))  # sort by 'duplicate_count' in descending order

# observation: the same car model with different(updated) prices are treated as different observations

In [ ]:
# aggregate the duplicates
select_columns = df.columns[df.columns.get_loc('country'):df.columns.get_loc('Wheelbase')+1]    # good way to select columns
df_agg = df.groupby(select_columns.tolist(), dropna=False).agg({'Price. (local)': 'mean', 'Price. (EUR)': 'mean', '2015':'sum', '2016':'sum', '2017':'sum', '2018':'sum', '2019':'sum', '2020':'sum', '2021':'sum', '2022':'sum', '2023':'sum'})
df_agg = df_agg.reset_index() 

# aggregate the duplicates without grouping by 'country'
select_columns = df.columns[df.columns.get_loc('Make Group'):df.columns.get_loc('Wheelbase')+1]
df_agg = df.groupby(select_columns.tolist(), dropna=False).agg({'Price. (local)': 'mean', 'Price. (EUR)': 'mean', '2015':'sum', '2016':'sum', '2017':'sum', '2018':'sum', '2019':'sum', '2020':'sum', '2021':'sum', '2022':'sum', '2023':'sum'})
df_agg = df_agg.reset_index() 

In [ ]:
# To-do-list: need to aggreagate body group into 5 classes: Car(Coupe, Sedan, Hatchback), SUV, Convertible, MPV, Wagon
df['Body Group'].unique()

In [ ]:
df['Make Group'].nunique()   # 68 parent firms

df['Make'].nunique()   # 120 brands

df.groupby(['Make', 'Body Group', 'Fuel Type'], dropna=False).ngroups   # 1316 unique combinations of Brand, Body Group and Fuel Type

68